In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from queue import PriorityQueue
import heapq
import statistics
from collections import Counter

In [17]:
data = pd.read_csv("kendaraan_clean_pca.csv")
# data = data.sample(10000)

In [18]:
X = data.drop(columns=["Tertarik"])
y = data[["Tertarik"]]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

In [19]:
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(398857, 2) (99715, 2) (398857, 1) (99715, 1)


In [6]:
class KDNode:
    def __init__(self, points, y, left = None, right = None, distance = np.inf):
        self.points = points
        self.y = y
        self.left = left
        self.right = right
        self.distance = distance
    def __lt__(self, other):
        return self.distance < other.distance

In [7]:
class KDTree:
    def __init__(self, k = 2, p = 2):
        self.tree = None
        self.k = k
        self.depth = 0
        self.heap = []
        self.p = p
    
    def fit(self, X, y):
        self.X = X
        self.y = y

        if isinstance(X, pd.DataFrame):
            X = X.values
        
        if isinstance(y, pd.DataFrame):
            y = y.values

        self.tree = self._construct_tree(np.array(X), np.array(y).reshape(-1), 0)
    
    def _construct_tree(self, points, y, depth):
        if len(points) == 0:
            return None
        
        k = len(points[0])
        axis = depth % k
        
        sort_by_axis = np.argsort(points[:, axis])
        sorted_points = points[sort_by_axis]
        sorted_y =  y[sort_by_axis]
        mid = len(sorted_points) // 2

        return KDNode(
            sorted_points[mid],
            sorted_y[mid],
            self._construct_tree(sorted_points[:mid], sorted_y[:mid], depth + 1),
            self._construct_tree(sorted_points[mid + 1:],sorted_y[mid + 1:], depth + 1)
        )
  
    def nearest_neighbour_search(self, query_point):
        k = len(query_point)
        heap = []
        
        def search(node, depth):
            if node == None:
                return

            nonlocal heap
            
            d = np.linalg.norm(query_point - node.points, ord = self.p)
            node.distance = -d
            heapq.heappush(heap, node)
            if len(heap) < self.k:
                heapq.heappush(heap, node)
            else:
                heapq.heappushpop(heap, node)

            axis = depth % k

            if query_point[axis] < node.points[axis]:
                close, other = node.left, node.right
            else:
                close, other = node.right, node.left

            search(close, depth + 1)

            delta = abs(query_point[axis] - node.points[axis])
            nearest = abs(heap[-1].distance)
            isFull = len(heap) > self.k

            if len(heap) < self.k or delta < nearest:
                search(other, depth + 1)

        search(self.tree, 0)
        return heap

    def predict(self, X_test):
        results = []
        if isinstance(X_test, pd.DataFrame):
            X_test = X_test.values
            
        for test in X_test:
            result = self.nearest_neighbour_search(test)
            predict_values = [item.y for item in result]
            counter = Counter(predict_values)
            results.append(counter.most_common(1)[0][0])
        return results

In [20]:
kdtree = KDTree(k = 2)
kdtree.fit(X_train, y_train)

In [21]:
y_pred = kdtree.predict(X_test)

In [23]:
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report

precision = accuracy_score(y_pred, y_test) * 100
class_report = classification_report(y_pred, y_test)
print("Accuracy with K-NN: {0:.2f}%".format(precision))
print(class_report)
print(confusion_matrix(y_pred, y_test))

Accuracy with K-NN: 80.74%
              precision    recall  f1-score   support

           0       0.70      0.89      0.79     39595
           1       0.91      0.76      0.83     60120

    accuracy                           0.81     99715
   macro avg       0.81      0.82      0.81     99715
weighted avg       0.83      0.81      0.81     99715

[[35093  4502]
 [14708 45412]]


In [24]:
# df_test = pd.read_csv("kendaraan_clean_pca_test.csv")
# y_pred_2_scratch = kdtree.predict(df_test.drop(columns=["Tertarik"]))

In [25]:
# precision3 = accuracy_score(y_pred_2_scratch, df_test[["Tertarik"]]) * 100
# class_report3 = classification_report(y_pred_2_scratch, df_test[["Tertarik"]])
# print("Accuracy with K-NN: {0:.2f}%".format(precision3))
# print(class_report3)
# print(confusion_matrix(y_pred_2_scratch, df_test[["Tertarik"]]))

In [26]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=2, p=2, algorithm='kd_tree')
knn.fit(X_train, y_train)
y_pred_1 = knn.predict(X_test)

C:\Users\milha\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\neighbors\_classification.py:179: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)


In [27]:
precision1 = accuracy_score(y_pred_1, y_test) * 100
class_report = classification_report(y_pred_1, y_test)
print("Accuracy with K-NN: {0:.2f}%".format(precision1))
print(class_report)
print(confusion_matrix(y_pred_1, y_test))

Accuracy with K-NN: 93.58%
              precision    recall  f1-score   support

           0       0.88      0.99      0.93     44642
           1       0.99      0.90      0.94     55073

    accuracy                           0.94     99715
   macro avg       0.94      0.94      0.94     99715
weighted avg       0.94      0.94      0.94     99715

[[44021   621]
 [ 5780 49293]]


In [28]:
df_test = pd.read_csv("kendaraan_clean_pca_test.csv")
y_pred_2 = knn.predict(df_test.drop(columns=["Tertarik"]))

In [29]:
precision2 = accuracy_score(y_pred_2, df_test[["Tertarik"]]) * 100
class_report2 = classification_report(y_pred_2, df_test[["Tertarik"]])
print("Accuracy with K-NN: {0:.2f}%".format(precision2))
print(class_report2)
print(confusion_matrix(y_pred_2, df_test[["Tertarik"]]))

Accuracy with K-NN: 98.65%
              precision    recall  f1-score   support

           0       0.98      1.00      0.99    244451
           1       1.00      0.98      0.99    254121

    accuracy                           0.99    498572
   macro avg       0.99      0.99      0.99    498572
weighted avg       0.99      0.99      0.99    498572

[[243493    958]
 [  5793 248328]]
